In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import bmra_prep
import bmra_prep.pathway_activity.prediction

In [2]:
data_dir = "01_outputs_2020"
out_dir = "02_outputs_2020"
info_dir = "DATA"

os.makedirs(out_dir, exist_ok = True)

# Load Data

In [3]:
# load metdadata dict and extract used elements
with open(os.path.join(data_dir, "metadata.pickle"), "rb") as f:
    all_metadata = pickle.load(f)

n_modules = all_metadata["n_modules"]
n_genes = all_metadata["n_genes"]
n_experiments = all_metadata["n_experiments"]

modules = all_metadata["modules"]
exp_ids = all_metadata["exp_ids"]
genes = all_metadata["genes"]

In [4]:
# load data
L1000_df = pd.read_csv(
    os.path.join(data_dir, "L1000_LFC_data.csv"),
    index_col = 0,
)

x = L1000_df.values
x.shape


(978, 263)

In [5]:
# load doses and perturbation matrix
inhib_conc_matrix = pd.read_csv(
    os.path.join(data_dir, "inhib_conc_annotated.csv"),
    index_col = 0,
).values
display(inhib_conc_matrix.shape)

ic50_matrix = pd.read_csv(
    os.path.join(data_dir, "ic50_annotated.csv"),
    index_col = 0,
).values
display(ic50_matrix.shape)


gamma_matrix = pd.read_csv(
    os.path.join(data_dir, "gamma_annotated.csv"),
    index_col = 0,
).values
display(gamma_matrix.shape)


pert_matrix = pd.read_csv(
    os.path.join(data_dir, "pert_annotated.csv"),
    index_col = 0,
).values
display(pert_matrix.shape)


(10, 263)

(10, 263)

(10, 263)

(10, 263)

In [6]:
y_true = (1 + gamma_matrix * inhib_conc_matrix / ic50_matrix) / (1 + inhib_conc_matrix / ic50_matrix)
display(y_true.shape)
y_true

(10, 263)

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.00990099, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 0.83660138,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

## Run models

In [7]:
a_coeffs = bmra_prep.pathway_activity.prediction.predict_coeffs(
    x, y_true, pert_matrix, 200_000,10,10,10,100)

In [8]:
#help(bmra_prep.pathway_activity.prediction.predict_coeffs)

In [9]:
a_coeffs_df = pd.DataFrame(a_coeffs,index=modules,columns = genes)
a_coeffs_df

AARS         ABCB6         ABCC5         ABCF1  \
CDK1/2    7.040280e-05 -1.011166e-05 -5.219648e-05  1.808735e-05   
CDK4/6    4.921144e-06  1.247889e-05 -9.695450e-06 -4.188825e-07   
IKK      -5.000214e-06 -1.763978e-05 -6.420929e-06  6.590868e-06   
TLR4     -2.164760e-07  1.863594e-06 -1.430741e-05  2.484297e-06   
mTOR     -1.971574e-05 -5.184613e-06 -2.289007e-07 -1.221543e-05   
PAK       2.156375e-02 -1.116969e-05 -3.786562e-06 -7.684908e-06   
ERK       4.757687e-06 -3.836366e-05  8.924221e-06 -9.403241e-06   
JAK/STAT -2.640747e-07 -4.375971e-06  4.387936e-05  3.554535e-06   
PLK1      3.412928e-05 -2.769096e-07 -2.606935e-04 -7.008038e-06   
ITK       1.624915e-05 -4.718382e-07 -3.679074e-06  7.565935e-06   

                 ABCF3     ABHD4         ABHD6          ABL1         ACAA1  \
CDK1/2    1.321535e-05  0.000022 -4.270415e-05  7.720674e-05  1.577360e-05   
CDK4/6    4.463274e-06 -0.000013 -2.960514e-06  2.118380e-07  4.484955e-06   
IKK      -5.644674e-06 -0.000013  9.716069e-06  2.283260e-06 -3.400610e-07   
TLR4      1.667942e-06 -0.000008 -2.961982e-06 -2.792228e-07  3.584781e-06   
mTOR     -9.227058e-07  0.000010  4.075444e-05  2.300727e-06 -3.473850e-05   
PAK      -8.137919e-06  0.000015 -8.591910e-06  4.327847e-06 -1.392315e-05   
ERK      -1.709477e-05  0.000004 -6.764451e-07  1.073712e-05 -1.588450e-05   
JAK/STAT  7.522672e-06 -0.000023 -1.721258e-06 -3.356435e-06 -1.390880e-04   
PLK1      6.197645e-06 -0.000006  1.155765e-06  1.491841e-05  1.043031e-05   
ITK       2.030844e-05  0.000007 -2.146318e-05  1.223256e-05 -2.752390e-05   

                 ACAT2  ...         ZMIZ1         ZMYM2    ZNF131  \
CDK1/2    1.045826e-04  ...  1.894578e-08  1.685622e-05  0.000031   
CDK4/6   -1.792175e-06  ... -9.935611e-06 -3.179738e-06 -0.000007   
IKK      -1.152863e-07  ...  2.606311e-06  2.733049e-06  0.000008   
TLR4     -3.145771e-06  ...  1.702871e-06 -1.007983e-06 -0.000004   
mTOR     -2.859003e-05  ...  4.853555e-06 -3.964285e-06 -0.000005   
PAK      -1.177237e-06  ... -1.369696e-05 -4.912152e-07 -0.000003   
ERK       2.489131e-05  ... -3.561507e-05 -1.452766e-05 -0.031512   
JAK/STAT -4.955163e-04  ... -2.100471e-06  7.414110e-07  0.000006   
PLK1      1.184138e-05  ...  1.180030e-05  3.830244e-06  0.000011   
ITK      -4.495826e-06  ...  1.321359e-05 -1.575395e-05  0.000013   

                ZNF274        ZNF318        ZNF395        ZNF451  \
CDK1/2    1.317058e-05 -4.275440e-06 -1.302887e-06  5.727096e-05   
CDK4/6   -1.322970e-06  3.224894e-06  4.002905e-06  5.621329e-06   
IKK      -3.005696e-07 -7.263681e-06  1.542892e-05 -7.986477e-07   
TLR4     -4.533565e-07 -8.514710e-07 -9.561557e-07  6.875291e-06   
mTOR     -2.194723e-05  4.550498e-06 -8.264220e-06 -3.904512e-06   
PAK       1.442643e-05  5.719693e-06  7.423674e-07  2.819497e-06   
ERK      -3.474133e-06 -6.634072e-07  3.666679e-06  2.378797e-06   
JAK/STAT -1.546517e-05 -1.902923e-06  1.922970e-06 -1.378152e-06   
PLK1      2.148698e-05 -2.620772e-06 -5.003263e-06 -3.567724e-06   
ITK      -1.644970e-05  3.863901e-06 -4.495629e-06 -2.525411e-05   

                ZNF586        ZNF589          ZW10  
CDK1/2    9.486550e-03 -1.142690e-05 -4.316513e-06  
CDK4/6   -2.195590e-06  4.637915e-07  1.271551e-06  
IKK       4.709241e-06 -7.258412e-06  1.781965e-07  
TLR4      1.173808e-07  6.656121e-07  1.554717e-06  
mTOR     -5.129703e-06  2.777482e-06  3.023588e-06  
PAK       3.023868e-06  6.787654e-06  1.673325e-05  
ERK      -2.399079e-06  7.082401e-06  7.404190e-03  
JAK/STAT  4.132516e-06  7.229021e-06  9.826156e-06  
PLK1      6.773293e-06 -2.669641e-06 -3.033237e-05  
ITK       4.037094e-07 -3.718446e-06 -3.097276e-06  

[10 rows x 978 columns]

In [10]:
(np.abs(a_coeffs_df) > 1e-4).sum(axis='columns')

CDK1/2       74
CDK4/6       10
IKK          19
TLR4         66
mTOR         67
PAK          46
ERK         111
JAK/STAT     42
PLK1         25
ITK           5
dtype: int64

In [11]:
#a_coeffs_df = pd.DataFrame(a_coeffs,index=modules,columns = genes)
a_coeffs_df.to_csv(os.path.join(out_dir, "a_coeffs.csv"))
#a_coeffs_df = pd.read_csv('a_coeffs.csv',index_col=0)
#a_coeffs = a_coeffs_df.values
#display(a_coeffs_df.astype(bool).sum(axis='columns'))
#display(a_coeffs_df)

In [12]:
#pathway_activity = a_coeffs @ x
#pathway_activity.shape

In [13]:
R_global = bmra_prep.pathway_activity.calc_global_response_from_pathway_activity(
    bmra_prep.pathway_activity.calc_pathway_activity(x,a_coeffs),
    modules,
    L1000_df.columns
)
R_global_df = R_global.dataframe
display(R_global_df)

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
CDK1/2              0.374603           -0.137492           -0.533110   
CDK4/6             -0.118398           -0.027737            0.059306   
IKK                -0.189119           -0.170653           -1.757087   
TLR4                0.710553           -0.740479            0.321673   
mTOR               -1.974737           -1.961689           -0.279687   
PAK                 0.083215           -0.056729            0.259757   
ERK                -0.437640           -1.886647           -0.566214   
JAK/STAT            0.698184            0.424347            0.179488   
PLK1                0.080809           -0.255565            0.376429   
ITK                -0.306057            0.568791           -0.770360   

          CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
CDK1/2              0.065439            0.237383            0.284852   
CDK4/6              0.024034            0.085174           -0.424450   
IKK                 0.187261            0.253872           -0.237323   
TLR4                0.370013           -1.355629           -1.796879   
mTOR               -1.273997           -1.980374           -0.774240   
PAK                -0.195724           -0.347050           -0.085756   
ERK                -0.650670           -0.239301           -0.072744   
JAK/STAT           -0.090389           -0.733237           -0.590702   
PLK1               -0.152404           -0.144975            0.246421   
ITK                 0.079526            0.680750            0.587803   

          EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
CDK1/2              -0.167230            0.171545            0.144066   
CDK4/6               0.019402            0.071208            0.104277   
IKK                  0.236512            0.198977            0.067704   
TLR4                -0.619664           -1.974616           -1.900657   
mTOR                -1.623362           -1.526877           -1.061128   
PAK                  0.154775            0.184581            0.035799   
ERK                  0.249141           -0.090062           -0.058921   
JAK/STAT             0.146477           -1.713002           -0.165344   
PLK1                -0.162250            0.098107            0.145986   
ITK                 -0.151320            0.808088            0.704316   

          EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
CDK1/2               0.038893  ...             -0.052059   
CDK4/6               0.098970  ...             -0.198745   
IKK                  0.009282  ...              0.432883   
TLR4                -0.743977  ...             -0.169788   
mTOR                -0.756345  ...             -0.454862   
PAK                  0.045928  ...             -0.001570   
ERK                  0.003748  ...             -0.188757   
JAK/STAT            -0.329542  ...             -1.678740   
PLK1                -0.033686  ...              0.100728   
ITK                  0.013359  ...              0.487184   

          AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  \
CDK1/2               -0.072298              -1.854854               0.104884   
CDK4/6               -0.309989              -0.073639              -0.114822   
IKK                   0.470171               0.671279               0.092816   
TLR4                 -0.124031               0.598956               0.182503   
mTOR                  0.526461               0.711704              -0.368525   
PAK                   0.033059              -0.479381              -0.021239   
ERK                  -0.253064              -0.512933               0.051010   
JAK/STAT             -1.849269               1.181472              -1.060296   
PLK1                 -0.227022              -0.726150               0.003315   
ITK                   0.662892              -0.445331               0.060170   

          AICHI001_THP1_24H:E23  AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  

In [14]:
#LFC_PLCg = pd.read_csv(os.path.join(info_dir,"PLCg_Data_log_2020.csv"),index_col=0)

#R_global_df.drop(columns=LFC_PLCg.index,inplace=True)

R_global_df.to_csv(os.path.join(out_dir, "R_global_annotated.csv"))
display(R_global_df)

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
CDK1/2              0.374603           -0.137492           -0.533110   
CDK4/6             -0.118398           -0.027737            0.059306   
IKK                -0.189119           -0.170653           -1.757087   
TLR4                0.710553           -0.740479            0.321673   
mTOR               -1.974737           -1.961689           -0.279687   
PAK                 0.083215           -0.056729            0.259757   
ERK                -0.437640           -1.886647           -0.566214   
JAK/STAT            0.698184            0.424347            0.179488   
PLK1                0.080809           -0.255565            0.376429   
ITK                -0.306057            0.568791           -0.770360   

          CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
CDK1/2              0.065439            0.237383            0.284852   
CDK4/6              0.024034            0.085174           -0.424450   
IKK                 0.187261            0.253872           -0.237323   
TLR4                0.370013           -1.355629           -1.796879   
mTOR               -1.273997           -1.980374           -0.774240   
PAK                -0.195724           -0.347050           -0.085756   
ERK                -0.650670           -0.239301           -0.072744   
JAK/STAT           -0.090389           -0.733237           -0.590702   
PLK1               -0.152404           -0.144975            0.246421   
ITK                 0.079526            0.680750            0.587803   

          EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
CDK1/2              -0.167230            0.171545            0.144066   
CDK4/6               0.019402            0.071208            0.104277   
IKK                  0.236512            0.198977            0.067704   
TLR4                -0.619664           -1.974616           -1.900657   
mTOR                -1.623362           -1.526877           -1.061128   
PAK                  0.154775            0.184581            0.035799   
ERK                  0.249141           -0.090062           -0.058921   
JAK/STAT             0.146477           -1.713002           -0.165344   
PLK1                -0.162250            0.098107            0.145986   
ITK                 -0.151320            0.808088            0.704316   

          EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
CDK1/2               0.038893  ...             -0.052059   
CDK4/6               0.098970  ...             -0.198745   
IKK                  0.009282  ...              0.432883   
TLR4                -0.743977  ...             -0.169788   
mTOR                -0.756345  ...             -0.454862   
PAK                  0.045928  ...             -0.001570   
ERK                  0.003748  ...             -0.188757   
JAK/STAT            -0.329542  ...             -1.678740   
PLK1                -0.033686  ...              0.100728   
ITK                  0.013359  ...              0.487184   

          AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  \
CDK1/2               -0.072298              -1.854854               0.104884   
CDK4/6               -0.309989              -0.073639              -0.114822   
IKK                   0.470171               0.671279               0.092816   
TLR4                 -0.124031               0.598956               0.182503   
mTOR                  0.526461               0.711704              -0.368525   
PAK                   0.033059              -0.479381              -0.021239   
ERK                  -0.253064              -0.512933               0.051010   
JAK/STAT             -1.849269               1.181472              -1.060296   
PLK1                 -0.227022              -0.726150               0.003315   
ITK                   0.662892              -0.445331               0.060170   

          AICHI001_THP1_24H:E23  AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  